# 基于特征点的视觉里程计

## 一、介绍
基于特征点的视觉里程计需要先从图像中提取特征点，计算描述子，匹配。在获得匹配好的点对后，根据点对估计相机的运动。根据相机的不同可以分成以下三种方法
1. 2D-2D方法：当两个相机都为单目时，采用对极几何约束解决，即先求本质矩阵 $E = t^{\wedge}R$，再对本质矩阵分解得到外参(这种方法不能确定尺度，t相差一个比例因子)。
2. 3D-3D：当相机为RGBD相机，则根据匹配点对可以得到匹配好的三维点对，再用ICP算法获得相机外参。
3. 3D-2D：获得了3D点和它对应的投影位置，这是标准的PnP问题(Perspective N Points),可以采用一系列PnP求解算法求解。


## 二、特征点法VIO的优缺点
目前基于特征点的视觉里程计占据主流地位，但仍有以下几个缺点
1. 关键点的提取与描述子的计算非常耗时，SIFT在CPU上无法实时计算，ORB需要20ms的计算时长。若整个SLAM系统以30ms/帧的速度运行，则一大半的时间要花在计算特征点上。
2. 使用特征点时，可能忽略特征点外的信息。相对于图像几十万个像素，特征点只有几百个(比如使用Harris角点、Fast角点等)。
3. 对于纹理较少的区域，会发生特征缺失。

而要想改进特征点法，有两种思路
* 光流法(Optical Flow)：光流法仍需提取特征点，但不计算特征点的描述子和对描述子进行匹配，取而代之的是对特征点运动的跟踪(光流估计)。光流法估计相机运动时仍采用对极几何、PnP或ICP算法，因此依然要求特征点的可区别性。
* 直接法(Direct Method)：直接法可以根据图像的像素灰度信息估计相机的运动，不需要提取到的点必需是角点

总结：特征点法估计相机运动是把特征点当成三维空间中的**不动点**，根据它们在相机中的投影位置，通过最小化**重投影误差**(Reprojection error)优化相机运动。这也是为什么特征点法需要花费巨大的计算量在计算、匹配特征点。而直接法中并不需要知道点之间的对应关系，通过最小化**光度误差**(Photometric error)来求得它们。

## 三、参考资料
[1] 视觉SLAM十四讲——从理论到实践 第七、八章 